In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random


model_name = 'eryk-mazus/polka-1.1b'
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [2]:
 tokenizer.get_vocab().items()

dict_items([('дён', 22841), ('<0xB9>', 188), ('▁accurate', 16232), ('恐', 33670), ('▁nation', 5233), ('▁dataframe', 12205), ('▁dimension', 9927), ('报', 33733), ('▁sync', 16523), ('Db', 10234), ('os▁', 40028), ('чай', 19424), ('拌', 33739), ('(w▁', 43005), ('▁Sit', 21988), ('.▁Cie', 41229), ('ו', 30205), ('affen', 19593), (',▁a▁po', 39368), ('▁!==', 15639), ('▁icons', 27673), ('>:', 23917), ('dr', 7707), ('▁rejected', 22225), ('jed', 11370), ('▁Fig', 5104), ('▁take', 2125), ('▁trova', 25307), ('▁Id', 5163), ('▁construction', 7632), ('▁Список', 22497), ('są▁', 36654), ('▁sampling', 23460), ('such', 14565), ('Pattern', 17144), ('gre', 7979), ('▁fruit', 15774), ('▁Walk', 12878), ('▁pani▁', 42113), ('竟', 34374), ('▁randomly', 20459), ('akces', 40715), ('▁Monday', 27822), ('▁ј', 12887), ('opol', 13242), ('▁Cle', 21386), ('icación', 29009), ('▁League', 5165), ('▁pleasant', 21246), ('▁connection', 3957), ('aces', 3302), ('akcja▁', 43489), ('▁Camb', 9287), ('tygodni', 41411), ('▁bei', 2862), ('准'

In [2]:
# def restricted_sampling(sentence_txt, predicate):
#     # Generate logits for the next token
#     with torch.no_grad():
#         input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
#         outputs = model(input_ids)
#         next_token_logits = outputs.logits[:, -1, :]  # Get logits for the last token

#     # Convert logits to probabilities
#     probs = F.softmax(next_token_logits, dim=-1).squeeze()

#     # Get tokens that start with "p" or "P" and set their probability to 0
#     for token_id in range(len(probs)):
#         token = tokenizer.decode(token_id)

#         if not predicate(token):
#             probs[token_id] = 0

#     # Renormalize probabilities
#     probs /= probs.sum()

#     # Sample from the filtered distribution
#     filtered_token_id = torch.multinomial(probs, num_samples=1).item()
#     return tokenizer.decode([filtered_token_id])

# def extend_sentence(sentence_txt, iter=10):
# 	letter = sentence_txt[0]

# 	for i in range(iter):
# 		if sentence_txt[-1] == " ":
# 			sentence_txt += restricted_sampling(sentence_txt, lambda token: token.startswith(letter))
# 		else:
# 			sentence_txt += restricted_sampling(sentence_txt, lambda token: token.startswith(" " + letter) or not token.startswith(" "))
			
# 	return sentence_txt

In [4]:
extend_sentence("piekarz piecze")

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
2024-10-30 00:33:14.186445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 00:33:14.197485: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 00:33:14.200281: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 00:33:14.208909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX

'piekarz pieczepieczeń-pełnych-przez-gluten'

In [15]:
from transformers import LogitsProcessor

class StrictStartingLetterProcessor(LogitsProcessor):
    def __init__(self, tokenizer, letter):
        self.tokenizer = tokenizer
        self.letter = letter.lower()

    def __call__(self, input_ids, scores):
        last_token_id = input_ids[0, -1].item()
        last_token = self.tokenizer.decode([last_token_id]).strip().lower()
        
        # if not last_token.startswith(self.letter):
        for i in range(scores.size(1)):
            token = self.tokenizer.decode([i]).strip().lower()
            if not token.startswith(self.letter):
                scores[0, i] = -float("inf")
        return scores

def generate_sentence_with_custom_logits(prefix, max_length, top_k, top_p, temperature):
    letter = prefix[0].lower()
    input_ids = tokenizer(prefix, return_tensors="pt")['input_ids'].to(device)
    
    # Dodanie CustomLogitsProcessor do generacji
    logits_processor = [StrictStartingLetterProcessor(tokenizer, letter)]
    
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=1.8,
        do_sample=True,
        logits_processor=logits_processor,  # Dodajemy modyfikator logitów
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.capitalize() + "."

def generate_best_sentence_with_filter(prefix, max_length, top_k, top_p, temperature, n_variants=2):
    candidates = [
        generate_sentence_with_custom_logits(prefix, max_length, top_k, top_p, temperature)
        for _ in range(n_variants)
    ]
    
    # Wybór najlepszego zdania, np. najdłuższego lub najbardziej spójnego
    best_sentence = max(candidates, key=len)
    return best_sentence


In [23]:
generate_best_sentence_with_filter(
	top_k = 5,
	top_p = 0.9,
	temperature = 0.7,
	max_length = 20,
	prefix = "Prawdziwy piekarz przyprawia"
)

'Prawdziwy piekarz przyprawia pierp przy pomocy palpit przepypu.'

In [24]:
generate_best_sentence_with_filter(
	top_k = 5,
	top_p = 0.9,
	temperature = 0.7,
	max_length = 20,
	prefix = "Nie należy natomiast"
)

'Nie należy natomiast nad ni nie na nognie nas naknu nas nie napnij.'

In [25]:
generate_best_sentence_with_filter(
	top_k = 5,
	top_p = 0.9,
	temperature = 0.7,
	max_length = 20,
	prefix = "Coraz częściej cukiernie"
)

'Coraz częściej cukiernie czy ci chcą ccc ccoc c.'